In [1]:
import os
CURRENT_DIR = os.getcwd()
PARENT_DIR = os.path.dirname(CURRENT_DIR)
print("Current Directory:", CURRENT_DIR)
print("Parent Directory:", PARENT_DIR)

Current Directory: /Users/selmanipek/GitHub/BrainTumorDetection/notebooks
Parent Directory: /Users/selmanipek/GitHub/BrainTumorDetection


In [2]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from icecream import ic
import sys
sys.path.append(PARENT_DIR)
from libs.preprocessor import Preprocessor
from libs.utils import *

In [3]:
from gtda.images import Binarizer
import numpy as np

# Binary classification

In [4]:
print("Generating topological data")
PATH = os.getcwd()
INPUT_TRAIN_PATH = os.path.join(PARENT_DIR,'data/interim/resized/Training/glioma') # Input path
OUTPUT_TRAIN_PATH = os.path.join(PARENT_DIR,'data/tda/Training/glioma/grayscale')

INPUT_TEST_PATH = os.path.join(PARENT_DIR,'data/interim/resized/Testing/glioma') # Input path
OUTPUT_TEST_PATH = os.path.join(PARENT_DIR,'data/tda/Testing/glioma/grayscale')

# create output folder if it doesn't exist
if not os.path.exists(OUTPUT_TRAIN_PATH):
    os.makedirs(OUTPUT_TRAIN_PATH)
if not os.path.exists(OUTPUT_TEST_PATH):
    os.makedirs(OUTPUT_TEST_PATH)

Generating topological data


In [5]:
def convert_to_grayscale(input_image_path, output_image_path):
    # Open the image
    original_image = Image.open(input_image_path)

    # Convert the image to grayscale
    grayscale_image = original_image.convert("L")

    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_image_path), exist_ok=True)

    # Save the grayscale image
    grayscale_image.save(output_image_path)

In [6]:
# Create list of filenames:
train_filenames = [file for file in os.listdir(INPUT_TRAIN_PATH) if os.path.isfile(os.path.join(INPUT_TRAIN_PATH, file))]
test_filenames = [file for file in os.listdir(INPUT_TEST_PATH) if os.path.isfile(os.path.join(INPUT_TEST_PATH, file))]

In [7]:
# Generate grayscale for train files:
if not os.path.exists(OUTPUT_TRAIN_PATH):
    for filename in train_filenames:
        IN = os.path.join(INPUT_TRAIN_PATH,filename)
        OUT = os.path.join(OUTPUT_TRAIN_PATH,filename)
        convert_to_grayscale(IN, OUT)
    

In [8]:
# Generate grayscale for train files:
if not os.path.exists(OUTPUT_TEST_PATH):
    for filename in test_filenames:
        IN = os.path.join(INPUT_TEST_PATH,filename)
        OUT = os.path.join(OUTPUT_TEST_PATH,filename)
        convert_to_grayscale(IN, OUT)

In [101]:
def binarize_and_filtrate(input_path,output_path,filenames,threshold,filtration,save):

    # define output path:
    #output = os.path.join(output_path,"")
    output = os.path.join(output_path,"binarized")
    #filt_output = os.path.join(output_path,)

    # create output folder if it doesn't exist
    if not os.path.exists(output):
        os.makedirs(output)

    # define threshold for binarizer
    binarizer = Binarizer(threshold=threshold)

    # inputting image size manually, can change later:
    ims = [];

    for file in filenames:
        im = Image.open(os.path.join(input_path,file))
        im_np = np.array(im)
        im_frmt = im_np[None, :,:]
        im_bin = binarizer.fit_transform(im_frmt)

        ims.append(im_bin)
    
    # convert to np_array:
    ims_arry = np.array(ims)

    if save == True:
        file = "thresh_"+str(threshold)+".npy"
        output_file = os.path.join(output,file)
        np.save(output_file,ims_arry)
    
    return ims_arry
    

In [107]:
# generate binarized images for training set
train_bin_035 = create_binarized_images(OUTPUT_TRAIN_PATH,os.path.join(OUTPUT_TRAIN_PATH,os.pardir),train_filenames,0.35,True)
# generate binarized images for testing set
train_bin_035 = create_binarized_images(OUTPUT_TEST_PATH,os.path.join(OUTPUT_TEST_PATH,os.pardir),test_filenames,0.35,True)

In [112]:
image_binarized

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]])

In [108]:
from gtda.images import RadialFiltration

radial_filtration = RadialFiltration(center=np.array([20, 6]))
image_filtration = radial_filtration.fit_transform(image_binarized)

radial_filtration.plot(image_filtration, colorscale="jet")

In [16]:
from gtda.homology import CubicalPersistence

cubical_persistence = CubicalPersistence(n_jobs=-1)
image_cubical = cubical_persistence.fit_transform(image_filtration)

cubical_persistence.plot(image_cubical)

In [18]:
# Open the image using PIL
image = Image.open(os.path.join(PATH,'data/interim/resized/Testing/notumor/Te-no_0030.jpg'))

image_np = np.array(image)
image_formatted = image_np[None, :,:]

binarizer = Binarizer(threshold=0.35)
image_binarized = binarizer.fit_transform(image_formatted)

binarizer.plot(image_binarized)

In [19]:
radial_filtration = RadialFiltration(center=np.array([20, 6]))
image_filtration = radial_filtration.fit_transform(image_binarized)

radial_filtration.plot(image_filtration, colorscale="jet")

In [20]:
cubical_persistence = CubicalPersistence(n_jobs=-1)
image_cubical = cubical_persistence.fit_transform(image_filtration)

cubical_persistence.plot(image_cubical)

In [21]:
# data/interim/Testing/meningioma/Te-me_0019.jpg

# Open the image using PIL
image = Image.open(os.path.join(PATH,'data/interim/resized/Testing/meningioma/Te-me_0019.jpg'))

image_np = np.array(image)
image_formatted = image_np[None, :,:]

binarizer = Binarizer(threshold=0.35)
image_binarized = binarizer.fit_transform(image_formatted)

binarizer.plot(image_binarized)

radial_filtration = RadialFiltration(center=np.array([20, 6]))
image_filtration = radial_filtration.fit_transform(image_binarized)

radial_filtration.plot(image_filtration, colorscale="jet")

In [22]:
cubical_persistence = CubicalPersistence(n_jobs=-1)
image_cubical = cubical_persistence.fit_transform(image_filtration)

cubical_persistence.plot(image_cubical)

In [23]:
#data/interim/Testing/notumor/Te-no_0032.jpg

# Open the image using PIL
image = Image.open(os.path.join(PATH,'data/interim/resized/Testing/notumor/Te-no_0032.jpg'))

image_np = np.array(image)
image_formatted = image_np[None, :,:]

binarizer = Binarizer(threshold=0.35)
image_binarized = binarizer.fit_transform(image_formatted)

binarizer.plot(image_binarized)

radial_filtration = RadialFiltration(center=np.array([20, 6]))
image_filtration = radial_filtration.fit_transform(image_binarized)

radial_filtration.plot(image_filtration, colorscale="jet")

In [24]:
# Create DataFrames with metadata of the preprocessed data
train_df = get_metadata(os.path.join(PATH,'data/interim/resized/Training', 'train'))
test_df = get_metadata(os.path.join(PATH,'data/interim/resized/Testing', 'test'))

NameError: name 'get_metadata' is not defined